%%writefile creation_db.hql

DROP DATABASE IF EXISTS demodb CASCADE;

%%writefile -a creation_db.hql
CREATE DATABASE demodb LOCATION '/user/jovyan/somemetastore';

! hive -f creation_db.hql

In [16]:
%%writefile external_table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;
DROP TABLE IF EXISTS posts_sample;
CREATE EXTERNAL TABLE posts_sample_external (
id int,
posttypeid int,
creationdate string,
tags string,
owneruserid int,
parentid int,
score int,
favoritecount int
)
ROW FORMAT
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
"input.regex" = '^\<row(?=.*\\bId=\"(\\d+))(?=.*\\bPostTypeId=\"(\\d+))(?=.*\\bCreationDate=\"([0-9-T:.]+)\")(?=.*\\bTags=\"([\\S\\s]+)\")?(?=.*\bOwnerUserId=\"(\\d+))?(?=.*\\bParentId=\"(\\d+))?(?=.*\\bScore=\"(\\d\+))?(?=.*\\bFavoriteCount =\"(\\d+))?.*$'
)
LOCATION '/data/stackexchange1000/posts/';

CREATE TABLE posts_sample (
id int,
posttypeid int,
creationdate string,
tags string,
owneruserid int,
parentid int,
score int,
favoritecount int
)
PARTITIONED BY (year STRING,month STRING)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;



Overwriting external_table.hql


In [17]:
%%writefile -a external_table.hql
SET hive.exec.dynamic.partition = true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=2000;
set hive.exec.max.dynamic.partitions.pernode=1000;
INSERT OVERWRITE TABLE posts_sample partition(year,month)
Select 
id ,
posttypeid ,
creationdate ,
tags ,
owneruserid ,
parentid ,
score ,
favoritecount,
substr(creationdate,1,4) as year,
substr(creationdate,1,7) as month
from posts_sample_external ;
with 
B as
(
    select year,month,cnt,row_number() over(order by month asc) as rn from A 
),
A as
(select year,month,count(*) as cnt from posts_sample group by year,month )
select year,month,cnt from B where rn=3;

Appending to external_table.hql


In [ ]:
! hive -f external_table.hql 



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 1.073 seconds
OK
Time taken: 1.626 seconds
OK
Time taken: 3.146 seconds
OK
Time taken: 0.652 seconds
OK
Time taken: 0.299 seconds
Query ID = jovyan_20180228062222_2d57c3a4-e997-4105-aa9c-d04745da0f71
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1519792833153_0020, Tracking URL = http://f860572437fc:8088/proxy/application_1519792833153_0020/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1519792833153_0020
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2018-02-28 06:22:47,644 Stage-1 map = 0%,  reduce = 0%
2018-02-28 06:23:06,157 Stage-1 map = 26%,  reduce = 0%, Cumulati